In [1]:
import pandas as pd
import numpy as np
import time
from tqdm.autonotebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

#from src.detect_peaks import detect_peaks
#from src.utils import *

from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

#импорт необходимых библиотек
import yfinance as yf
from pycoingecko import CoinGeckoAPI

import requests

import pandas_datareader as pd_reader

import tsfel

/var/folders/rh/7rjcc_rs68n6wjb41vhqg3dh0000gn/T/ipykernel_86232/2111625055.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## ----------------------------------------------------------------------------------------------- ##  
# Задание 1
## ----------------------------------------------------------------------------------------------- ##  

1) Получаем данные

2) Определяем количество кластеров

3) Проводим нормализацию

4) Проводим кластеризацию методом Dynamic Time Warping

In [2]:
#get TOP 100 crypto 
url = 'https://coin360.com/coin'
r = requests.get(url)
crypto = pd.read_html(r.text)

crypto = list(crypto[0]['Symbol'])

for i in range(len(crypto)):
    crypto[i] = crypto[i].lower()

In [3]:
rem = []

rem.append('wbnb')
rem.append('xcn')
rem.append('bttc')
rem.append('iota')
rem.append('bnx')
rem.append('wemix')

for i in rem:
    crypto.remove(i)

In [4]:
#collect data

import cryptocompare as cc
import datetime

def get_minute_data(date=datetime.datetime.now()):
    coins_df = []
    try:
        for i in crypto:
            x = cc.get_historical_price_minute(i, currency='USD', limit=1440, toTs=date)
            x = pd.DataFrame(x).fillna(method='ffill')
            x = x.drop(columns=['conversionSymbol', 'conversionType', 'volumefrom'], errors='ignore')
            x['time'] = pd.to_datetime(x['time'],unit='s')
            x['ticker'] = i
            x = np.array(x)
            coins_df.append(x)
        return(np.array(coins_df))
    except:
        pass

def get_week_data(date=datetime.datetime.now()):
    coins_df = []
    try:
        for i in crypto:
            x = cc.get_historical_price_hour(i, currency='USD', limit=168, toTs=date)
            x = pd.DataFrame(x).fillna(method='ffill')
            x = x.drop(columns=['conversionSymbol', 'conversionType', 'volumefrom'], errors='ignore')
            x['time'] = pd.to_datetime(x['time'],unit='s')
            x['ticker'] = i
            x = np.array(x)
            coins_df.append(x)
        return(np.array(coins_df))
    except:
        pass

def get_day_data(date=datetime.datetime.now()):
    coins_df = []
    try:
        for i in crypto:
            x = cc.get_historical_price_day(i, currency='USD', limit=60, toTs=date)
            x = pd.DataFrame(x).fillna(method='ffill')
            x = x.drop(columns=['conversionSymbol', 'conversionType', 'volumefrom'], errors='ignore')
            x['time'] = pd.to_datetime(x['time'],unit='s')
            x['ticker'] = i
            x = np.array(x)
            coins_df.append(x)
        coins_df = np.array(coins_df)
        return(np.array(coins_df))
    except:
        pass

In [5]:
minutes = get_minute_data()
weeks = get_week_data()
days = get_day_data()

[ERROR] CCCAGG market does not exist for this coin pair (GMX-USD)
[ERROR] CCCAGG market does not exist for this coin pair (GMX-USD)
[ERROR] CCCAGG market does not exist for this coin pair (GMX-USD)


In [6]:
all_data = [minutes, weeks, days]

In [7]:
#fromatting and normalisation 

from sklearn import preprocessing

def formatting_and_norm(dttm, norm=1):
    new_min = []
    min_max_scaler = preprocessing.MinMaxScaler()
    if norm == 1:
        for i in range(len(dttm)):
            coin = pd.DataFrame(dttm[i])
            coin.set_index([0], inplace=True)
            coin = coin.T[:-1]
            coin = np.array(coin)
            coin = min_max_scaler.fit_transform(coin)
            new_min.append(coin)
    else:
        for i in range(len(dttm)):
            coin = pd.DataFrame(dttm[i])
            coin.set_index([0], inplace=True)
            coin = coin.T[:-1]
            coin = np.array(coin)
            new_min.append(coin)
    return(np.array(new_min))

In [8]:
#K-Means clustering 

def clustr(dttm, k_range):

    distortions = []
    K = k_range
    
    appended = pd.DataFrame()
    for i in formatting_and_norm(dttm):
        appended = pd.concat([appended, pd.DataFrame(i)])
  

    for k in tqdm(K):
        kmeanModel = KMeans(n_clusters=k)
        kmeanModel.fit(appended)
        distortions.append(kmeanModel.inertia_)

    plt.figure(figsize=(12,6))
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('Elbow Method')
    plt.show()


In [9]:
clustr(minutes, range(2,16))

TypeError: object of type 'NoneType' has no len()

In [ ]:
clustr(days, range(2,16))

In [ ]:
clustr(weeks, range(2,16))

In [ ]:
def appended_df(dttm):
    appended = pd.DataFrame()
    for i in formatting_and_norm(dttm, 1):
        appended = pd.concat([appended, pd.DataFrame(i)])
    return(appended)

def appended_df_no_norm(dttm):
    appended = pd.DataFrame()
    for i in formatting_and_norm(dttm, 0):
        appended = pd.concat([appended, pd.DataFrame(i)])
    return(appended)

In [ ]:
from tslearn.clustering import TimeSeriesKMeans, silhouette_score

def kmeans(dttm, n=10, form=1):
    
    appended = pd.DataFrame()
    for i in formatting_and_norm(dttm, form):
        appended = pd.concat([appended, pd.DataFrame(i)])
  
    
    distortions = []
    silhouette = []
    K = range(2, n)
    for k in tqdm(K):
        kmeanModel = TimeSeriesKMeans(n_clusters=k, metric="euclidean", n_jobs=6, max_iter=10, n_init=5)
        kmeanModel.fit(appended)
        distortions.append(kmeanModel.inertia_)
        silhouette.append(silhouette_score(appended, kmeanModel.labels_, metric="euclidean"))

    fig, ax1 = plt.subplots(figsize=(8, 6))

    ax2 = ax1.twinx()
    ax1.plot(K, distortions, 'b-')
    ax2.plot(K, silhouette, 'r-')

    ax1.set_xlabel('# clusters')
    ax1.set_ylabel('Distortion', color='b')
    ax2.set_ylabel('Silhouette', color='r')
    plt.figure(figsize=(9,6))
    plt.show()

In [ ]:
kmeans(minutes), kmeans(days), kmeans(weeks)
kmeans(minutes, 10, 0), kmeans(days, 10, 0), kmeans(weeks, 10, 0)

**Имеем:**

С нормировкой:

Для минутных данных - 3 кластера

Для дневных данных - 3 кластера

Для недельных данных - 3 кластера

Без нормировки:

Для минутных данных - 3 кластера

Для дневных данных - 3 кластера

Для недельных данных - 3 кластера

In [ ]:
def clusters_show(dttm, n_clusters=3, form=1):
    ts_kmeans = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", n_jobs=10, max_iter=10)
    if form == 1:
        ts_kmeans.fit(appended_df(dttm))
    else:
        ts_kmeans.fit(appended_df_no_norm(dttm))

    for cluster_number in range(n_clusters):
        plt.plot(ts_kmeans.cluster_centers_[cluster_number, :, 0].T, label=cluster_number)
    plt.title("Cluster centroids")
    plt.legend()
    plt.show()

In [ ]:
clusters_show(minutes, 3), clusters_show(days, 6), clusters_show(weeks, 3)

## ----------------------------------------------------------------------------------------------- ##  
# Задание 2
## ----------------------------------------------------------------------------------------------- ##  

А) Используем данные с прошлого д/з  

Б) Пробуем tsfresh, tsfel  

В) Применяем k-means ( нужно передать feature matrix на вход k-means)  

Г) Визуализируем результат  

Д) Пишем сравнительный анализ с фич экстрактором и без него (5 - 10 предложений)  

In [ ]:
def clean_dataset(df):
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return(df[indices_to_keep].astype(np.float64))

In [ ]:
from tsfresh import extract_features

def prep_for_fresh(df):
    appended = pd.DataFrame()
    for i, x in enumerate(df):
        x = pd.DataFrame(x)
        x = x.rename(columns={0: 'time', 1: 'price1', 2: 'price2', 3: 'price3', 4: 'price4', 5: 'price5', 6: 'name'})
        x = x.drop(['time'], axis=1)
        try:
            x = x.drop(['name'], axis=1)
        except:
            pass
        x = x.astype('float')
        x['time'] = list(range(0, len(x)))
        x['coin'] = i
        appended = pd.concat([appended, x])
    return(appended)

def features(dttm):
    new_prices_scaled_tsfresh_allf = extract_features(prep_for_fresh(dttm), 
                                                      column_id = "coin", 
                                                      column_sort = "time")
    df = new_prices_scaled_tsfresh_allf.dropna(axis='columns')
    df = clean_dataset(df)
    K = range(2,15)
    distortions = []
    for k in tqdm(K):
        kmeanModel = KMeans(n_clusters=k)
        kmeanModel.fit(df)
        distortions.append(kmeanModel.inertia_)

    plt.figure(figsize=(12,6))
    plt.plot(K, distortions, 'bx-')
    plt.xlabel('k')
    plt.ylabel('Distortion')
    plt.title('Elbow Method')
    plt.show()
    

In [ ]:
features(minutes), features(weeks), features(days)

In [ ]:
#отдельно сформируем функцию для вывода фичей из tsfel

import tsfel

from IPython.display import clear_output


# библиотека tsfel
def get_tsfel_features(dttm):
    cfg_file = tsfel.get_features_by_domain('spectral')
    tsfel_data = pd.DataFrame()

    for i in range(len(dttm)):
        x = pd.DataFrame(minutes[i])[[1,2,3,4,5]]
        print(i)
        new = tsfel.time_series_features_extractor(cfg_file, x, fs= 100)
        clear_output(wait=True)
        tsfel_data = pd.concat([tsfel_data, new])

    tsfel_data = tsfel_data.reset_index(drop=True).dropna(axis='columns')
    return(tsfel_data)

In [ ]:
plt.figure(figsize=(18,11))
plt.tight_layout(pad=5.0)

#визуализация k-NN без предобработки
plt.subplot(4,1,1)

ts_kmeans = TimeSeriesKMeans(n_clusters=3, metric="euclidean", n_jobs=10, max_iter=10)
ts_kmeans.fit(appended_df_no_norm(minutes)[::5].reset_index(drop=True).T)

for cluster_number in range(3):
    plt.plot(pd.DataFrame(ts_kmeans.cluster_centers_[cluster_number]), label = cluster_number)

plt.xlabel('Currency',  fontsize = 20)
plt.ylabel('Distance to cluster', color='b',  fontsize = 20)
plt.title('without_extraction_raw_data', fontsize = 20)


#визуализация k-NN с нормализацией 
plt.subplot(4,1,2)

ts_kmeans = TimeSeriesKMeans(n_clusters=3, metric="euclidean", n_jobs=10, max_iter=10)
ts_kmeans.fit(appended_df(minutes)[::5].reset_index(drop=True).T)

for cluster_number in range(3):
    plt.plot(pd.DataFrame(ts_kmeans.cluster_centers_[cluster_number]), label = cluster_number)

plt.xlabel('Currency',  fontsize = 20)
plt.ylabel('Distance to cluster', color='b',  fontsize = 20)
plt.title('without_extraction_normalised_data', fontsize = 20)


#визуализация кластеров после обработки tsfresh
plt.subplot(4,1,3)

df = extract_features(prep_for_fresh(minutes), column_id = "coin", column_sort = "time")
df = df.dropna(axis='columns')

ts_kmeans = TimeSeriesKMeans(n_clusters=3, metric="euclidean", n_jobs=10, max_iter=10)
ts_kmeans.fit(df.T)

for cluster_number in range(3):
    plt.plot(pd.DataFrame(ts_kmeans.cluster_centers_[cluster_number]), label = cluster_number)

plt.xlabel('Currency',  fontsize = 20)
plt.ylabel('Distance to cluster', color='b',  fontsize = 20)    
plt.title('after_tsfresh_extraction', fontsize = 20)


#визуализация кластеров после обработки tsfel
plt.subplot(4,1,4)

ts_kmeans = TimeSeriesKMeans(n_clusters=3, metric="euclidean", n_jobs=10, max_iter=10)
ts_kmeans.fit(get_tsfel_features(minutes).T)

for cluster_number in range(3):
    plt.plot(pd.DataFrame(ts_kmeans.cluster_centers_[cluster_number]), label = cluster_number)

plt.xlabel('Currency',  fontsize = 20)
plt.ylabel('Distance to cluster', color='b',  fontsize = 20)    
plt.title('after_tsfel_extraction', fontsize = 20)


plt.legend()
plt.show()

**Сравнительный анализ:**
    
1) При кластеризации сырых данных не видно явное присутсвие других кластеров. 
Можно сказать, что применение кластеризации к сырым данным не имеет смысла.

2) Примение кластеризации к нормированным данным уже позволяет выделить существенные группы валют. 
Но есть предположение, что при нормализации данных MinMaxScaller валюты кластеризовались относительно своих цен, а не трендов.

3) Кластеризация с использованием фич-экстракторов на первый взгляд слабо отличается друг от друга. 
На вход подавались сырые данные. Необходимо проверить методы с использованием нормализации.

In [ ]:
def re_tsfresh(dttm):
    x = prep_for_fresh(dttm)
    x = extract_features(x, column_id = "coin", column_sort = "time")

    #time, coin = list(x['time']), list(x['coin'])
    #x = x[['price1', 'price2', 'price3', 'price4', 'price5']]

    min_max_scaler = preprocessing.MinMaxScaler()
    
    x = pd.DataFrame(min_max_scaler.fit_transform(x))
    #x = x.rename(columns={0: 'time', 1: 'price1', 2: 'price2', 3: 'price3', 4: 'price4'})
    #x['time'] = time
    #x['coin'] = coin
    return(x)

def re_tsfel(dttm):
    x = get_tsfel_features(dttm)
    x = clean_dataset(x.dropna(axis='columns'))
    min_max_scaler = preprocessing.MinMaxScaler()
    x = pd.DataFrame(min_max_scaler.fit_transform(x))
    return(x)

In [ ]:
#визуализация кластеров после обработки tsfresh и нормализации
plt.figure(figsize=(18,11))
plt.subplot(2,1,1)

#df = extract_features(re_tsfresh(minutes), column_id = "coin", column_sort = "time")
#df = df.dropna(axis='columns')
#df = df.drop(columns=['time', 'coin'])

ts_kmeans = TimeSeriesKMeans(n_clusters=3, metric="euclidean", n_jobs=10, max_iter=10)
ts_kmeans.fit(re_tsfresh(minutes).T)

for cluster_number in range(3):
    plt.plot(pd.DataFrame(ts_kmeans.cluster_centers_[cluster_number]), label = cluster_number)

plt.xlabel('Currency',  fontsize = 20)
plt.ylabel('Distance to cluster', color='b',  fontsize = 20)    
plt.title('after_tsfresh_extraction_and_norm', fontsize = 20)


#визуализация кластеров после обработки tsfel и нормализации
plt.subplot(2,1,2)

plt.tight_layout(pad=8.0)


ts_kmeans = TimeSeriesKMeans(n_clusters=3, metric="euclidean", n_jobs=10, max_iter=10)
ts_kmeans.fit(re_tsfel(minutes).T)

for cluster_number in range(3):
    plt.plot(pd.DataFrame(ts_kmeans.cluster_centers_[cluster_number]), label = cluster_number)

plt.xlabel('Currency',  fontsize = 20)
plt.ylabel('Distance to cluster', color='b',  fontsize = 20)    
plt.title('after_tsfel_extraction_and_norm', fontsize = 20)


plt.legend()
plt.show()

**Сравнительный анализ:**
    
1) Кластеризация после tsfresh и нормализации плохо отражена из-за масштаба. 
На промежутках, где нет пиков - расстояние до центров кластеров, как будто, одинаковое для всех монет. 
Плохое качество клстеризации.

2) TSFEL показал результаты лучше - кластеры выделились однозначно. 
Визуально понятно, к какому кластеру относится та или иная валюта. 
Расстояния хорошо представими.

## ----------------------------------------------------------------------------------------------- ##  
# Задание 3
## ----------------------------------------------------------------------------------------------- ##  

А) Используем данные с прошлого д/з  

Б) Применяем DTW, визуализируем  

В) Применяем иерархический DTW, визуализируем  

Г) Пишем сравнительные анализ иерархического и нормального DTW. Чем отличаются методы?  

Д) Что такое DTW distance? Как оно рассчитывается?  

In [ ]:
n_clusters = 5
ts_kmeans_dtw = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", n_jobs=6, max_iter=10)
ts_kmeans_dtw.fit(re_tsfresh(minutes).T)

for cluster_number in range(n_clusters):
    plt.plot(ts_kmeans_dtw.cluster_centers_[cluster_number, :, 0].T, label=cluster_number)
plt.title("Cluster centroids",  fontsize = 20)
plt.legend()
plt.show()

In [ ]:
n_clusters = 5
ts_kmeans_dtw = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", n_jobs=6, max_iter=10)
ts_kmeans_dtw.fit(re_tsfel(minutes).T)

for cluster_number in range(n_clusters):
    plt.plot(ts_kmeans_dtw.cluster_centers_[cluster_number, :, 0].T, label=cluster_number)
plt.title("Cluster centroids",  fontsize = 20)
plt.legend()
plt.show()

In [ ]:
# иерархический dtw

# Custom Hierarchical clustering
plt.figure(figsize=(18,11))
plt.subplot(4,1,1)

model1 = clustering.Hierarchical(dtw.distance_matrix_fast, {})
cluster_idx = model1.fit(new_prices_scaled)
shc.dendrogram(cluster_idx)


# Keep track of full tree by using the HierarchicalTree wrapper class
plt.subplot(4,1,2)
model2 = clustering.HierarchicalTree(model1)
cluster_idx = model2.fit(new_prices_scaled)
shc.dendrogram(cluster_idx)

# You can also pass keyword arguments identical to instantiate a Hierarchical object
plt.subplot(4,1,3)
model2 = clustering.HierarchicalTree(dists_fun=dtw.distance_matrix_fast, dists_options={})
cluster_idx = model2.fit(new_prices_scaled)
shc.dendrogram(cluster_idx)

# SciPy linkage clustering
plt.subplot(4,1,4)
dtw_cluster = clustering.LinkageTree(dtw.distance_matrix_fast, {})
cluster_idx = dtw_cluster.fit(new_prices_scaled)
shc.dendrogram(cluster_idx)

plt.show()

Что такое DTW distance?  
украл из леции:  
$$DTW(x, y) = min_{\pi} \sqrt{\sum_{(i,j)\in \pi} d(x_i, y_j)^2},$$

где $\pi = [\pi_0, ..., \pi_K]$ - это путь, удовлетворяющий набору условий:
- это лист парных индексов $\pi_k = (i_k, j_k)$, где  $0\leq i_k < n$ и $0\leq j_k < m$
- $\pi_0 = (0, 0)$ и $\pi_K = (n-1, m-1)$
- для всех $k>0$, $\pi_k=(i_k, j_k)$ и $\pi_{k-1}=(i_{k-1}, j_{k-1})$ удовлетворяют следующим неравенствам:
    - $i_{k-1} \leq i_k \leq i_{k-1} + 1$
    - $j_{j-1} \leq j_k \leq j_{k-1} + 1$  
      
Своими словами:  